In [1]:
import numpy as np
from numpy.linalg import solve

In [2]:
def getPi(n, j, k, kr):
  kkr = k - kr #指定クラスを1引いたもの
  if min(kkr) < 0:
    return 0
  if j == 0 and sum(kkr) == 0: #Initializationより
    return 1
  if j == 1 and sum(kkr) == 0: #Initializationより
    return 0
  if j == 0 and sum(kkr) >0: #(8.45)
    #print('(8.45)')
    state_number = int(getState(K, kkr))
    sum_emlam = 0
    for _r in range(len(K)):
      sum_emlam += alpha[n][_r] * Lambda[_r][state_number] / mu[n][_r]
    sum_pi = 0
    for _j in range(1, m[n]):
      #print('call getPi8_44({0}, {1}, {2}, {3})'.format(n, _j, kkr, kr))
      sum_pi += (m[n] - _j ) * getPi8_44(n, _j, kkr, kr) #このgetPiは人数を減らさない
    #print('getPi8_44 = {0}'.format(getPi8_44(n, _j, kkr, kr)))
    #print('8.45 sum_pi = {0}, (n, j) = ({1}, {2})'.format(sum_pi, n, j))
    return 1 - 1 / m[n] * (sum_emlam + sum_pi)
  if j > 0 and sum(kkr) > 0: #(8.44) また未チェック
    #print('(8.44)!!')
    sum_val = 0
    state_number = int(getState(K, kkr))
    for _r in range(len(K)):
      sum_val += alpha[n][_r] * Lambda[_r][state_number] * Lambda[_r][state_number] / mu[n][_r] * getPi(n, j-1, kkr, kr)
    return 1 / j * (sum_val) 

In [3]:
def getPi8_44(n, j, k, kr): #(8.45)から(8.44)を呼び出すときだけ利用 (人数を減らさず呼び出し)
  #print('getPi8_44({0}, {1}, {2}, {3})'.format(n, j, k, kr))
  sum_val = 0
  state_number = int(getState(K, k))
  for _r in range(len(K)):
    kr = np.zeros(len(k))
    kr[_r] += 1
    sum_val += alpha[n][_r] * Lambda[_r][state_number] / mu[n][_r] * getPi(n, j-1, k, kr) #ここで呼び出すgetPiのkrを変更しないといけない->修正、値があった
  #print('getPi8_44 = {0}'.format(sum_val))
  return 1 / j * (sum_val) 

In [4]:
def getState_kr(K, k, kr):
  state = 0
  kkr = k - kr
  if min(kkr) < 0:
    return -1
  else:
    for i in range(len(k)):
      #state += k[i] * (max(K) + 1)**int(len(k)-1-i)
      state += kkr[i] * (max(K) + 1)**int(len(kkr)-1-i)
    return state

In [5]:
def getState(K, k):
  state = 0
  for i in range(len(k)):
    state += k[i] * (max(K) + 1)**int(len(k)-1-i)
  return state

In [6]:
def getL(n, r, state_number):
  if state_number >= 0:
    return L[n,r,state_number]
  else:
    return 0

In [7]:
def calcT(k, state_number):
  for r in range(len(K)):
    #現在の状態と、クラスから1引いたkrを求め、k-1rの状態を持ってくる
    #state_number = getState(K, k) #現在の状態
    kr = np.zeros(len(k))
    kr[r] += 1
    kr_state_number = int(getState_kr(K, k, kr)) #対象クラスから1引いた状態

    if kr_state_number < 0:
      for n in range(len(m)):
        T[n,r,state_number] = 0
    else:
      for n in range(len(m)): # r = 0
        if m[n] >= 1:
          sum_L = 0
          for _r in range(len(K)): #Lの和の部分を計算
            sum_L += getL(n, _r, kr_state_number)
          T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + sum_L)
          if m[n] > 1:
            sum_pi = 0
            for _j in range(m[n]-2+1):
              sum_pi += getPi(n, _j, k, kr) 
            T[n,r,state_number] += 1 / (m[n] * mu[n,r]) * sum_pi
        if m[n] == 0:
          T[n,r,state_number] = 1 / mu[n,r]

In [8]:
def calcLambda(k, state_number):
  for r in range(len(K)):#クラス数ループ
    et = 0
    for i in range(len(m)):#ノード数ループ
      et += alpha[i,r] * T[i,r,state_number]
    if et == 0:
      continue
    Lambda[r][state_number] = k[r] / et

In [9]:
def calcL(k, state_number):
  for i in range(len(m)):#ノード数ループ
    for r in range(len(K)):#クラス数ループ
      L[i,r,state_number] = Lambda[r][state_number] * T[i,r,state_number] * alpha[i,r]

In [10]:
#初期設定
K = [2, 1]
m = [2, 1, 0] #Node3はIS(Infinite server)
mu = np.array([[5.0, 5.0], [5/2, 5/3], [1, 0.5]])
alpha = np.array([[1.0, 1.0], [0.6, 0.3], [0.4, 0.7]])

#2022/07/11 状態算出 (今回は3進数2桁)
max_state_number = max(K) * (max(K) + 1)**1 + max(K) * (max(K) + 1)**0
max_state_number

L = np.zeros((len(m), len(K), max_state_number))
T = np.zeros((len(m), len(K), max_state_number))
Lambda = np.zeros((len(K), max_state_number))

k_list = np.array([[1, 0], [0, 1], [1, 1], [2, 0], [2, 1]])

In [12]:
for k in k_list:
  print(k)
  state_number = getState(K, k)
  print('state_number = {0}'.format(state_number))
  calcT(k, state_number)
  print('T')
  print(T)
  calcLambda(k, state_number)
  print('Lambda')
  print(Lambda)
  calcL(k, state_number)
  print('L')
  print(L)

[1 0]
state_number = 3
T
[[[0.  0.  0.  0.2 0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0. ]]

 [[0.  0.  0.  0.4 0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0. ]]

 [[0.  0.  0.  1.  0.  0.  0.  0. ]
  [0.  0.  0.  0.  0.  0.  0.  0. ]]]
Lambda
[[0.         0.         0.         1.19047619 0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
L
[[[0.         0.         0.         0.23809524 0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]]

 [[0.         0.         0.         0.28571429 0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]]

 [[0.         0.         0.         0.47619048 0.         0.
   0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.        ]]]
[0 1]
state_number = 1
T
[[[0.  0.  0.  0.2 0

In [13]:
#k = np.array([1, 0])

#for r in range(len(K)):

#  state_number = getState(K, k)
#  kr = np.zeros(len(k))
#  kr[r] += 1
#  kr_state_number = int(getState_kr(K, k, kr))
#  print(kr_state_number)

#  if kr_state_number < 0:
#    for n in range(len(m)):
#      T[n,r,state_number] = 0
#  else:
#    for n in range(len(m)): # r = 0
#      if m[n] >= 1:
#        sum_L = 0
#        for _r in range(len(K)): #Lの和の部分を計算
#          sum_L += getL(n, _r, kr_state_number)
#        T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + sum_L)
#        if m[n] > 1:
#          sum_pi = 0
#          for _j in range(m[n]-2+1):
#            sum_pi += getPi(n, _j, k, kr) 
#          T[n,r,state_number] += 1 / (m[n] * mu[n,r]) * sum_pi
          #T[n,r,state_number] += 1 / (m[n] * mu[n,r]) * getPi(n, 0, k, kr) #この0は一般化する
      #if m[n] > 1:
      #  T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + getL(n,0,kr_state_number) + getL(n,1,kr_state_number) + getPi(n, 0, k, kr)) #getLの和を作成
      #if m[n] == 1:
      #  T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + getL(n,0,kr_state_number) + getL(n,1,kr_state_number)) 
#      if m[n] == 0:
#        T[n,r,state_number] = 1 / mu[n,r]   


#r = 1
#kr = np.zeros(len(k))
#kr[r] += 1
#print(k - kr)
#print(min(k - kr))
#kr_state_number = int(getState_kr(K, k, kr))
#print(kr_state_number)
#k - krを実施して、最小値が負になる場合は計算をしないようにする

#if kr_state_number < 0:
#  for n in range(len(m)):
#    T[n,1,state_number] = 0
#else: 
#  for n in range(len(m)): 
#    if m[n] > 1:
#      T[n,1,state_number] = 1 / (m[n] * mu[n,1]) * (1 + getL(n,0,kr_state_number) + getL(n,1,kr_state_number) + getPi(n, 0, k, kr))
#    if m[n] == 1:
#      T[n,1,state_number] = 1 / (m[n] * mu[n,1]) * (1 + getL(n,0,kr_state_number) + getL(n,1,kr_state_number)) 
#    if m[n] == 0:
#      T[n,1,state_number] = 1 / mu[n,1]   


#for n in range(len(m)):
#  if m[n] > 1:
#    T[n,0,1,0] = 1 / (m[n] * mu[n,0]) * (1 + L[n,0,1-1,0] + L[n,1,1-1,0] + getPi(n, 0, [0, 0]))
#  if m[n] == 1:
#    T[n,0,1,0] = 1 / (m[n] * mu[n,0]) * (1 + L[n,0,1-1,0] + L[n,1,1-1,0]) 
#  if m[n] == 0:
#    T[n,0,1,0] = 1 / mu[n,0]   
#T[1,0,1,0] = 1 / (m[1] * mu[1,0]) * (1 + L[1,0,1-1,0] + L[1,1,1-1,0])
#T[2,0,1,0] = 1 / mu[2,0]
#T

In [14]:
#Step2.2
#et = 0
#for i in range(len(m)):
#  et += alpha[i,0] * T[i,0,state_number]
  #et += alpha[i,0] * T[i,0,1,0]
#Lambda[0][state_number] = k[0] / et
#Lambda[0] = k[0] / (alpha[0,0] * T[0,0,1,0] + alpha[1,0] * T[1,0,1,0] + alpha[2,0] * T[2,0,1,0])
#Lambda

In [15]:
#Step2.3
#L[0,0,state_number] = Lambda[0][state_number] * T[0,0,state_number] * alpha[0,0]
#L[1,0,state_number] = Lambda[0][state_number] * T[1,0,state_number] * alpha[1,0]
#L[2,0,state_number] = Lambda[0][state_number] * T[2,0,state_number] * alpha[2,0]

#L[0,0,1,0] = Lambda[0] * T[0,0,1,0] * alpha[0,0]
#L[1,0,1,0] = Lambda[0] * T[1,0,1,0] * alpha[1,0]
#L[2,0,1,0] = Lambda[0] * T[2,0,1,0] * alpha[2,0]
#L

In [16]:
#k = np.array([0, 1])
#for r in range(len(K)):
#  state_number = getState(K, k)
#  kr = np.zeros(len(k))
#  kr[r] += 1
#  kr_state_number = int(getState_kr(K, k, kr))
#  print(kr_state_number)

#  if kr_state_number < 0:
#    for n in range(len(m)):
#      T[n,r,state_number] = 0
#  else:
#    for n in range(len(m)): 
#      if m[n] >= 1:
#        sum_L = 0
#        for _r in range(len(K)): #Lの和の部分を計算
#          sum_L += getL(n, _r, kr_state_number)
#        T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + sum_L)
#        if m[n] > 1:
#          sum_pi = 0
#          for _j in range(m[n]-2+1):
#            sum_pi += getPi(n, _j, k, kr) 
#          T[n,r,state_number] += 1 / (m[n] * mu[n,r]) * sum_pi
#      if m[n] == 0:
#        T[n,r,state_number] = 1 / mu[n,r]   
#T

In [17]:
#k = [0, 1]
#state_number = k[0] * (max(K) + 1)**1 + k[1] * (max(K) + 1)**0
#print(state_number)

#for n in range(len(m)):
#  if m[n] > 1:
#    T[n,1,state_number] = 1 / (m[n] * mu[n,1]) * (1 + L[n,0,0] + L[n,1,0] + getPi(n, 0, [0, 0]))
#  if m[n] == 1:
#    T[n,1,state_number] = 1 / (m[n] * mu[n,1]) * (1 + L[n,0,0] + L[n,1,0]) 
#  if m[n] == 0:
#    T[n,1,state_number] = 1 / mu[n,1]


#for n in range(len(m)):
#  if m[n] > 1:
#    T[n,1,0,1] = 1 / (m[n] * mu[n,1]) * (1 + L[n,0,0,1-1] + L[n,1,0,1-1] + getPi(n, 0, [0, 0]))
#  if m[n] == 1:
#    T[n,1,0,1] = 1 / (m[n] * mu[n,1]) * (1 + L[n,0,0,1-1] + L[n,1,0,1-1]) 
#  if m[n] == 0:
#    T[n,1,0,1] = 1 / mu[n,1]
#T

In [18]:
#Step2.2
#et = 0
#for i in range(len(m)):
#  et += alpha[i,1] * T[i,1,state_number]
  #et += alpha[i,1] * T[i,1,0,1]
#Lambda[1][state_number] = k[1] / et
#Lambda[0] = k[0] / (alpha[0,0] * T[0,0,1,0] + alpha[1,0] * T[1,0,1,0] + alpha[2,0] * T[2,0,1,0])
#Lambda

In [19]:
#Step2.3
#L[0,1,state_number] = Lambda[1][state_number] * T[0,1,state_number] * alpha[0,1]
#L[1,1,state_number] = Lambda[1][state_number] * T[1,1,state_number] * alpha[1,1]
#L[2,1,state_number] = Lambda[1][state_number] * T[2,1,state_number] * alpha[2,1]

#L[0,1,1,0] = Lambda[1] * T[0,1,0,1] * alpha[0,1]
#L[1,1,1,0] = Lambda[1] * T[1,1,0,1] * alpha[1,1]
#L[2,1,1,0] = Lambda[1] * T[2,1,0,1] * alpha[2,1]
#L

In [20]:
#k = np.array([1, 1])
#for r in range(len(K)):
#  state_number = getState(K, k)
#  kr = np.zeros(len(k))
#  kr[r] += 1
#  kr_state_number = int(getState_kr(K, k, kr))
#  print(kr_state_number)

#  if kr_state_number < 0:
#    for n in range(len(m)):
#      T[n,r,state_number] = 0
#  else:
#    for n in range(len(m)): 
#      if m[n] >= 1:
#        sum_L = 0
#        for _r in range(len(K)): #Lの和の部分を計算
#          sum_L += getL(n, _r, kr_state_number)
#        print('sum_L = {0}, (n, r) = ({1}, {2})'.format(sum_L, n, r))
#        T[n,r,state_number] = 1 / (m[n] * mu[n,r]) * (1 + sum_L)
#        if m[n] > 1:
#          sum_pi = 0
#          for _j in range(m[n]-2+1):
#            sum_pi += getPi(n, _j, k, kr)
#            print('getPi({0}, {1}, {2}, {3}) = {4}'.format(n, _j, k, kr, getPi(n, _j, k, kr))) 
#          print('sum_pi = {0}, (n, r) = ({1}, {2})'.format(sum_pi, n, r))
#          T[n,r,state_number] += 1 / (m[n] * mu[n,r]) * sum_pi
#      if m[n] == 0:
#        T[n,r,state_number] = 1 / mu[n,r]   
#T
#getPi8_44が２回呼び出されている

In [21]:
#Step2.2
#for r in range(len(K)):
#  et = 0
#  for i in range(len(m)):
#    et += alpha[i,r] * T[i,r,state_number]
    #et += alpha[i,1] * T[i,1,0,1]
#  Lambda[r][state_number] = k[r] / et
  #Lambda[0] = k[0] / (alpha[0,0] * T[0,0,1,0] + alpha[1,0] * T[1,0,1,0] + alpha[2,0] * T[2,0,1,0])
#Lambda

In [22]:
#Step2.3
#for n in range(len(m)):
#  for r in range(len(K)):
#    L[n,r,state_number] = Lambda[r][state_number] * T[n,r,state_number] * alpha[n,r]
#L